In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [41]:
train_df = pd.read_csv("../input/comp90086-2021/COMP90086_2021_Project_train/train.csv")
train_df

,id,x,y
0,IMG2744_1,-9.380678,3.58272
1,IMG2744_2,-9.380678,3.58272
2,IMG2744_3,-9.380678,3.58272
3,IMG2744_4,-9.380678,3.58272
4,IMG2744_5,-9.380678,3.58272
...,...,...,...
7495,IMG4243_1,-4.680678,35.18272
7496,IMG4243_2,-4.680678,35.18272
7497,IMG4243_3,-4.680678,35.18272
7498,IMG4243_4,-4.680678,35.18272


In [42]:
# convert to tuples to pass labels to tf datasets
targets = train_df.iloc[:, 1:3].apply(tuple, axis = 1)
targets

0        (-9.380678081, 3.58271965)
1        (-9.380678081, 3.58271965)
2        (-9.380678081, 3.58271965)
3        (-9.380678081, 3.58271965)
4        (-9.380678081, 3.58271965)
                   ...             
7495    (-4.680678082, 35.18271965)
7496    (-4.680678082, 35.18271965)
7497    (-4.680678082, 35.18271965)
7498    (-4.680678082, 35.18271965)
7499    (-4.680678082, 35.18271965)
Length: 7500, dtype: object

In [43]:
batch_size = 32
img_height = 680
img_width = 490

data_dir = pathlib.Path("../input/comp90086-2021/COMP90086_2021_Project_train")
data_dir

PosixPath('../input/comp90086-2021/COMP90086_2021_Project_train')

In [44]:
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.model_selection import train_test_split


#scaler = MinMaxScaler()
#target_scaler = scaler.fit(train_df.iloc[:, 1:3])

#targets = target_scaler.transform(train_df.iloc[:, 1:3])

In [45]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels=targets.tolist(),
  validation_split=0.2,
  subset="training",
  seed = 123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 7500 files belonging to 1 classes.
Using 6000 files for training.


In [46]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels=targets.tolist(),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 7500 files belonging to 1 classes.
Using 1500 files for validation.


In [47]:
for images, labels in train_ds.take(1):  # only take first element of dataset
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()

In [48]:
numpy_labels

array([[  43.31932  ,    9.982719 ],
       [   9.619322 ,   24.78272  ],
       [ -66.18068  ,   86.88272  ],
       [-127.980675 ,  110.88272  ],
       [  64.11932  ,  -66.31728  ],
       [-101.78068  ,  107.18272  ],
       [ -69.78068  ,  107.08272  ],
       [  67.41932  ,  -24.51728  ],
       [  73.31932  ,  -33.01728  ],
       [-158.18068  ,   81.98272  ],
       [-107.88068  ,   93.58272  ],
       [ -43.380676 ,   44.68272  ],
       [  52.619324 ,  -13.61728  ],
       [  -2.280678 ,   25.08272  ],
       [  14.719322 ,   25.48272  ],
       [ -49.68068  ,   44.082718 ],
       [ -86.38068  ,  120.28272  ],
       [  64.31932  ,  -29.617281 ],
       [  47.31932  ,  -20.21728  ],
       [  50.31932  ,    1.4827197],
       [ -85.38068  ,   56.68272  ],
       [  67.019325 ,  -23.617281 ],
       [  66.31932  ,  -64.41728  ],
       [  -8.480679 ,  -22.91728  ],
       [  52.719322 ,    2.7827196],
       [ -42.280678 ,   46.68272  ],
       [  52.619324 ,  -51.71728  ],
 

In [49]:
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [50]:
from tensorflow.keras import optimizers
model.compile(loss='mean_squared_error', optimizer=optimizers.RMSprop(lr=1e-4))

In [ ]:
history = model.fit(train_ds, epochs=20, batch_size = batch_size, validation_data = val_ds)

Epoch 1/20
188/188 [==============================] - 78s 408ms/step - loss: 1706.4658 - val_loss: 1789.5510
Epoch 2/20
188/188 [==============================] - 78s 406ms/step - loss: 1654.2590 - val_loss: 1719.9097
Epoch 3/20
188/188 [==============================] - 78s 408ms/step - loss: 1606.3221 - val_loss: 1975.7825
Epoch 4/20
188/188 [==============================] - 77s 405ms/step - loss: 1553.2928 - val_loss: 1782.4659
Epoch 5/20
 72/188 [==========>...................] - ETA: 40s - loss: 1507.9946

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
test_dir = pathlib.Path("../input/comp90086-2021/COMP90086_2021_Project_test")

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  label_mode=None,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
preds = model.predict(test_ds)

In [ ]:
preds

In [ ]:
import csv

test_filenames = []
with open("../input/comp90086-2021/COMP90086_2021_Project_test/imagenames.csv", newline='') as inputfile:
    reader = csv.reader(inputfile)
    # skip header
    header = next(reader)
    
    for row in reader:
        test_filenames.append(str(row[0]))
    
test_filenames[0]

In [ ]:
with open("submission.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["id","x","y"])
    index = 0
    for prediction in preds:
        writer.writerow([test_filenames[index], preds[index][0], preds[index][1]])
        index+=1